<a href="https://colab.research.google.com/github/RomainReina/Projet_NLP/blob/main/Projet_NLP_REINAROMAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sommaire
- [Introduction](#0)
- [Part 1:  Load de la data](#1)
    - [1.1  embeding](#1.1)
    - [1.2  Data generator](#1.2)
		- [Exercise 01](#ex01)
- [Part 2:  Word embedding](#2)
	- [Exercise 02](#ex02)
- [Part 3:  initialisation du modele](#3)
	- [Exercise 03](#ex03)
- [Part 4:  training](#4)


In [1]:
!pip -q install trax==1.3.1


     |████████████████████████████████| 348kB 4.3MB/s 
     |████████████████████████████████| 3.4MB 7.4MB/s 
     |████████████████████████████████| 1.5MB 42.3MB/s 
     |████████████████████████████████| 215kB 49.8MB/s 
     |████████████████████████████████| 983kB 40.3MB/s 
     |████████████████████████████████| 5.6MB 45.1MB/s 
     |████████████████████████████████| 368kB 54.6MB/s 
     |████████████████████████████████| 368kB 49.5MB/s 
     |████████████████████████████████| 655kB 22.6MB/s 
     |████████████████████████████████| 358kB 46.1MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 194kB 47.4MB/s 
     |████████████████████████████████| 706kB 35.0MB/s 
     |████████████████████████████████| 1.9MB 45.3MB/s 
     |████████████████████████████████| 3.8MB 53.8MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 1.2MB 50.1MB/s 
     |████████████████████████████████| 245kB 55.3MB/s

In [2]:
# Import Python libraries and helper functions 
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

import trax 
from trax import layers as tl

from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import nltk

import random

import matplotlib.pyplot as plt



nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

Mounted at /content/gdrive


In [5]:
csv = pd.read_csv('gdrive/MyDrive/Projet_NLP/training.1600000.processed.noemoticon.csv', encoding = 'latin', header=None)
csv

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [8]:
data = csv.drop([1,2,3,4],axis = 1)
data.columns = ['index', 'comments']
data


,index,comments
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [219]:

n = 200
#j'en veux que 200 + et 200 - 
df = data[:n]
negative_value = df.to_dict()
df = data[-n:]
positive_value = df.to_dict()


print(negative_value ,'\n', positive_value)

{'index': {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 137

In [244]:
#val_pos
df = data[n:n*2]
val_ng = df.to_dict()
df = data[-n*2:-n]
val_ps = df.to_dict()

In [255]:
neg = negative_value.get('comments')
print(neg)
pos = positive_value.get('comments')
print(pos)

val_ng = val_ng.get('comments')
val_ps = val_ps.get('comments')

dictionnaire = {}
for k in range(len(neg)):
  dictionnaire[k] = neg[k]
for k in range(len(pos)):
  dictionnaire[k+len(neg)]  = pos[k+1599800]
print(dictionnaire) 

{0: "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D", 1: "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", 2: '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds', 3: 'my whole body feels itchy and like its on fire ', 4: "@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. ", 5: '@Kwesidei not the whole crew ', 6: 'Need a hug ', 7: "@LOLTrish hey  long time no see! Yes.. Rains a bit ,only a bit  LOL , I'm fine thanks , how's you ?", 8: "@Tatiana_K nope they didn't have it ", 9: '@twittera que me muera ? ', 10: "spring break in plain city... it's snowing ", 11: 'I just re-pierced my ears ', 12: "@caregiving I couldn't bear to watch it.  And I thought the UA loss was embarrassing . . . . .", 13: '@octolinz16 It it counts, idk why I did either. you never talk t

In [256]:
neg_index_tmp = negative_value.get('index')
print(neg_index_tmp)
pos_index_tmp = positive_value.get('index')
print(pos_index_tmp)



{0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 137: 0, 138: 

In [257]:
def tokenize(data):


  for k, v in data.items():
    data[k] = re.sub('@[^\s]+','username', data[k])
    data[k] = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(data[k]).lower())


  #tokenization
  dict_data_token = dict()
  for k, v in data.items():
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(v)
    dict_data_token[k] = tokens

  #stopwords
  dict_data_stop = dict()
  for k, v in dict_data_token.items():
    dict_data_stop[k] = [w for w in v if not w in list(nltk.corpus.stopwords.words())]


  dict_data_stem = dict()
  st = LancasterStemmer()

  for k, v in dict_data_stop.items():
    dict_data_stem[k] = [st.stem(w) for w in v ]

  return dict_data_stop
  #return dict_data_stem



In [258]:
dictionnaire_tokenized = tokenize(dictionnaire)


In [259]:
print(f"The number of positive sentences: {len(dictionnaire_tokenized)}")
print(f"First sentences : {neg[0]}")
print(f"First sentences tokenize : {dictionnaire_tokenized[0]}")
print(" ")

#on converti en list
word_list=list(dictionnaire_tokenized.values())
word_list =sum(word_list,[])
print(word_list)

data_neg = list(neg.values())
data_pos = list(pos.values())
print(data_neg)
print(data_pos)

val_neg = list(val_ng.values())
val_pos = list(val_ps.values())
print(val_neg)

data2_pneg = list()

index_neg = list(neg_index_tmp.values())
index_pos = list(pos_index_tmp.values())



# on regarde si il n'y a rien en double

#nltk.FreqDist(word_list) # il y a des mot en double ! 


The number of positive sentences: 400
First sentences : @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
First sentences tokenize : ['username', 'awww', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day']
 
['username', 'awww', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day', 'upset', 'update', 'facebook', 'texting', 'might', 'cry', 'result', 'school', 'today', 'blah', 'username', 'dived', 'many', 'times', 'ball', 'managed', 'save', '50', 'rest', 'go', 'bounds', 'whole', 'body', 'feels', 'itchy', 'like', 'fire', 'username', 'behaving', 'mad', 'see', 'username', 'whole', 'crew', 'need', 'hug', 'username', 'hey', 'long', 'time', 'see', 'yes', 'rains', 'bit', 'bit', 'lol', 'fine', 'thanks', 'username', 'nope', 'username', 'muera', 'spring', 'break', 'plain', 'city', 'snowing', 'pierced', 'ears', 'username', 'bear', 'watch', 'thought', 'ua', 'loss', 'embarrassing', 'username', 'counts', 'idk', 'either',

In [250]:
#Creation du vocab

Vocab = {'__PAD__': 0, '__</e>__': 1, '__UNK__': 2} 

for word in word_list: 
  if word not in Vocab: 
     Vocab[word] = len(Vocab)
    
print("Total words in vocab are",len(Vocab))  #1493
# display(Vocab)

Total words in vocab are 1493


# converting into tensor

In [28]:
def toTensor(sentence, vocabulary = Vocab , unknowToken = '__UNK__'):
  '''
  Input: 
    sentence - String qui contien la phrase
    vocabulary - le dictionnaire vocab
    unknowToken - si le mot n'est pad connu, on met le label '__UNK__'
  Output:
    tensor - la liste avec des chiffres consernant les mots a analyser    
  '''  
  tokenizer = nltk.RegexpTokenizer(r'\w+')
  tensor = []
  # on tokenize la phrase
    
  tokened = tokenizer.tokenize(sentence)

  # get __UNK__ index 
  unk_ID = vocabulary.get(unknowToken, -1)  # = 2

  for word in tokened:
    word_ID = vocabulary.get(word, unk_ID)

    tensor.append(word_ID) 
  return tensor


In [29]:

print(f"sentence: {dictionnaire[0]} ")
print(f"sentence into tensor: {toTensor(dictionnaire[0])} ")

sentence: username awww that s a bummer you shoulda got david carr of third day to do it d 
sentence into tensor: [3, 4, 2, 2, 2, 5, 2, 6, 7, 8, 9, 2, 10, 11, 2, 2, 2, 2] 


# Data generator


In [30]:
def data_generator(data_pos, data_neg, batch_size, loop, vocab_dict, shuffle=False):
    '''
    Input: 
        data_pos - Set of positive examples
        data_neg - Set of negative examples
        batch_size - number of samples per batch. Must be even
        loop - True or False
        vocab_dict - The words dictionary
        shuffle - Shuffle the data order
    Yield:
        inputs - Subset of positive and negative examples
        targets - The corresponding labels for the subset
        example_weights - An array specifying the importance of each example
        
    '''     
### START GIVEN CODE ###
    # make sure the batch size is an even number
    # to allow an equal number of positive and negative samples
    assert batch_size % 2 == 0
    
    # Number of positive examples in each batch is half of the batch size
    # same with number of negative examples in each batch
    n_to_take = batch_size // 2
    
    # Use pos_index to walk through the data_pos array
    # same with neg_index and data_neg
    pos_index = 0
    neg_index = 0
    
    len_data_pos = len(data_pos)
    len_data_neg = len(data_neg)
    
    # Get and array with the data indexes
    pos_index_lines = list(range(len_data_pos))
    neg_index_lines = list(range(len_data_neg))
    
        
    stop = False
    
    # Loop indefinitely
    while not stop:  
        
        # create a batch with positive and negative examples
        batch = []
        
        # First part: Pack n_to_take positive examples
        
        # Start from pos_index and increment i up to n_to_take
        for i in range(n_to_take):
                    
            # If the positive index goes past the positive dataset lenght,
            if pos_index >= len_data_pos: 
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                
                # If user wants to keep re-using the data, reset the index
                pos_index = 0
                
                if shuffle:
                    # Shuffle the index of the positive sample
                    rnd.shuffle(pos_index_lines)
                    
            # get the tweet as pos_index
            tweet = data_pos[pos_index_lines[pos_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            
            tensor = toTensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment pos_index by one
            pos_index = pos_index + 1

### END GIVEN CODE ###
            
### START CODE HERE (Replace instances of 'None' with your code) ###

        # Second part: Pack n_to_take negative examples
    
        # Using the same batch list, start from neg_index and increment i up to n_to_take
        for i in range(n_to_take):
            
            # If the negative index goes past the negative dataset length,
            if neg_index >= len_data_neg:
                
                # If loop is set to False, break once we reach the end of the dataset
                if not loop:
                    stop = True;
                    break;
                    
                # If user wants to keep re-using the data, reset the index
                neg_index = 0
                
               
            # get the tweet as neg_index
            tweet = data_neg[neg_index_lines[neg_index]]
            
            # convert the tweet into tensors of integers representing the processed words
            tensor = toTensor(tweet, vocab_dict)
            
            # append the tensor to the batch list
            batch.append(tensor)
            
            # Increment neg_index by one
            neg_index += 1

### END CODE HERE ###        

### START GIVEN CODE ###
        if stop:
            break;

        # Update the start index for positive data 
        # so that it's n_to_take positions after the current pos_index
        pos_index += n_to_take
        
        # Update the start index for negative data 
        # so that it's n_to_take positions after the current neg_index
        neg_index += n_to_take
        
        # Get the max tweet length (the length of the longest tweet) 
        # (you will pad all shorter tweets to have this length)
        max_len = max([len(t) for t in batch]) 
        
        
        # Initialize the input_l, which will 
        # store the padded versions of the tensors
        tensor_pad_l = []
        # Pad shorter tweets with zeros
        for tensor in batch:
### END GIVEN CODE ###

### START CODE HERE (Replace instances of 'None' with your code) ###
            # Get the number of positions to pad for this tensor so that it will be max_len long
            n_pad = max_len - len(tensor)
            
            # Generate a list of zeros, with length n_pad
            pad_l = [0] * n_pad
            
            # concatenate the tensor and the list of padded zeros
            tensor_pad = tensor + pad_l
            
            # append the padded tensor to the list of padded tensors
            tensor_pad_l.append(tensor_pad)

        # convert the list of padded tensors to a numpy array
        # and store this as the model inputs
        inputs = np.array(tensor_pad_l)
  
        # Generate the list of targets for the positive examples (a list of ones)
        # The length is the number of positive examples in the batch
        target_pos = [1] * n_to_take
        
        # Generate the list of targets for the negative examples (a list of zeros)
        # The length is the number of negative examples in the batch
        target_neg = [0] * n_to_take
        
        # Concatenate the positve and negative targets
        target_l = target_pos + target_neg
        
        # Convert the target list into a numpy array
        targets = np.array(target_l)

        # Example weights: Treat all examples equally importantly.It should return an np.array. Hint: Use np.ones_like()
        example_weights = np.ones_like(targets)
        

### END CODE HERE ###

### GIVEN CODE ###
        # note we use yield and not return
        yield inputs, targets, example_weights

In [189]:
def index_generator(index_pos, index_neg):
  yield index_pos, index_neg

In [194]:
for k in range(len(index_pos)):
  index_pos[k] = str(index_pos[k])
for k in range(len(index_pos)):
  index_neg[k] = str(index_neg[k])

In [195]:

# Test the data_generator
batch_size=4
shuffle = False
# Create a data generator for training data,
# which produces batches of size 4 (for tensors and their respective targets)
tmp_data_gen = data_generator(data_pos, data_neg, batch_size, True, Vocab, shuffle)

# Call the data generator to get one batch and its targets
tmp_inputs, tmp_targets, tmp_example_weights = next(tmp_data_gen)

print(f"The inputs shape is {tmp_inputs.shape}")
print(f"The targets shape is {tmp_targets.shape}")
print(f"The example weights shape is {tmp_example_weights.shape}")

for i,t in enumerate(tmp_inputs):
    print(f"input tensor: {t}; target {tmp_targets[i]}; example weights {tmp_example_weights[i]}")

The inputs shape is (4, 22)
The targets shape is (4,)
The example weights shape is (4,)
input tensor: [  2 870   2   2   2  11   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]; target 1; example weights 1
input tensor: [  2 871 825 872 109   2 873   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0]; target 1; example weights 1
input tensor: [2 2 2 2 2 2 2 2 2 5 2 6 7 2 2 2 2 2 2 2 2 2]; target 0; example weights 1
input tensor: [ 2 12  2  2  2  2 13  2  2  2 15  2  2 16 17  2  2 18  2 20  2  2]; target 0; example weights 1


#Build Model

In [196]:
def SER(vocab_size=len(Vocab), embedding_dim=256, output_dim=2, mode='train'):

    '''
      Input: 
        vocab_size - integer containing the size of the vocabulary
        d_model - integer describing the embedding size
      Output:
        model - a trax serial model
    '''
     ### START CODE HERE (Replace instances of 'None' with your code) ###
    # create embedding layer
    embed_layer = tl.Embedding(
        vocab_size=vocab_size, # Size of the vocabulary
        d_feature=embedding_dim)  # Embedding dimension
    
    # Create a mean layer, to create an "average" word embedding
    mean_layer = tl.Mean(axis = 1)
    
    # Create a dense layer, one unit for each output
    dense_output_layer = tl.Dense(n_units = output_dim)

    
    # Create the log softmax layer (no parameters needed)
    log_softmax_layer = tl.LogSoftmax()
    
    # Use tl.Serial to combine all layers
    # and create the classifier
    # of type trax.layers.combinators.Serial
    model = tl.Serial(
      embed_layer, # embedding layer
      mean_layer, # mean layer
      dense_output_layer, # dense output layer 
      log_softmax_layer # log softmax layer
    )
### END CODE HERE ### 
    return model

In [197]:
# initializing your model
model = SER()
# display your model
print(model)

Serial[
  Embedding_1493_256
  Mean
  Dense_2
  LogSoftmax
]


##Train your model 

In [260]:
from trax.supervised import training

random.seed(33)

batch_size = 64
batch_size=4
shuffle = False

# Create training data, mask pad id=35180 for training.
train_generator = trax.supervised.inputs.add_loss_weights(
    data_generator(data_pos, data_neg, batch_size, True, Vocab, shuffle),
    id_to_mask=Vocab['__PAD__'])


# Create validation data, mask pad id=35180 for training.
eval_generator = trax.supervised.inputs.add_loss_weights(
    data_generator(val_pos, val_neg, batch_size, True, Vocab, shuffle),
    id_to_mask=Vocab['__PAD__'])



In [261]:
def train_model(SER, train_generator, eval_generator, train_steps=1, output_dir='model'):
    '''
    Input: 
        NER - the model you are building
        train_generator - The data generator for training examples
        eval_generator - The data generator for validation examples,
        train_steps - number of training steps
        output_dir - folder to save your model
    Output:
        training_loop - a trax supervised training Loop
    '''
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    train_task = training.TrainTask(
      train_generator, # A train data generator
      loss_layer = tl.CrossEntropyLoss(), # A cross-entropy loss function
      optimizer =  trax.optimizers.Adam(0.01),  # The adam optimizer
    )

    eval_task = training.EvalTask(
      labeled_data = eval_generator, # A labeled data generator
      metrics = [tl.CrossEntropyLoss(), tl.Accuracy()], # Evaluate with cross-entropy loss and accuracy
      n_eval_batches = 10 # Number of batches to use on each evaluation
    )

    training_loop = training.Loop(
        SER, # A model to train
        train_task, # A train task
        eval_task = eval_task, # The evaluation task
        output_dir = output_dir) # The output directory

    # Train with train_steps
    training_loop.run(n_steps = train_steps)
    ### END CODE HERE ###
    return training_loop

In [262]:
train_steps = 100        
!rm -f 'model/model.pkl.gz'  # Remove old model.pkl if it exists
model = SER()

# Train the model
training_loop = train_model(model, train_generator, eval_generator, 1000)

Step      1: train CrossEntropyLoss |  0.38607445
Step      1: eval  CrossEntropyLoss |  0.52961197
Step      1: eval          Accuracy |  0.65000000
Step    100: train CrossEntropyLoss |  0.01190062
Step    100: eval  CrossEntropyLoss |  0.00000002
Step    100: eval          Accuracy |  1.00000000
Step    200: train CrossEntropyLoss |  0.00000046
Step    200: eval  CrossEntropyLoss |  0.00000000
Step    200: eval          Accuracy |  1.00000000
Step    300: train CrossEntropyLoss |  0.00000048
Step    300: eval  CrossEntropyLoss |  0.00000000
Step    300: eval          Accuracy |  1.00000000
Step    400: train CrossEntropyLoss |  0.00000049
Step    400: eval  CrossEntropyLoss |  0.00000000
Step    400: eval          Accuracy |  1.00000000
Step    500: train CrossEntropyLoss |  0.00000049
Step    500: eval  CrossEntropyLoss |  0.00000000
Step    500: eval          Accuracy |  1.00000000
Step    600: train CrossEntropyLoss |  0.00000050
Step    600: eval  CrossEntropyLoss |  0.00000005


In [263]:
# ok ???

#Part 4: Compute Accuracy


In [264]:
# create the evaluation inputs
x, y,z = next(data_generator(data_pos, data_neg, batch_size, True, Vocab, shuffle))
print("input shapes", x.shape, y.shape,z.shape)

input shapes (4, 22) (4,) (4,)


In [265]:
tmp_pred = model(x)
print(type(tmp_pred))
print(f"tmp_pred has shape: {tmp_pred.shape}")

<class 'jax.interpreters.xla._DeviceArray'>
tmp_pred has shape: (4, 2)


In [266]:
def evaluate_prediction(pred, labels, pad):
    """
    Inputs:
        pred: prediction array with shape 
            (num examples, max sentence length in batch, num of classes)
        labels: array of size (batch_size, seq_len)
        pad: integer representing pad character
    Outputs:
        accuracy: float
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
## step 1 ##
    outputs = np.argmax(pred, axis=-1)
    print("outputs shape:", outputs.shape)

## step 2 ##
    mask = ~(labels == pad)
    print("mask shape:", mask.shape, "mask:", mask)
## step 3 ##
    accuracy = np.sum(outputs == labels) / np.sum(mask)
    ### END CODE HERE ###
    return accuracy

In [267]:
accuracy = evaluate_prediction(model(x), y, Vocab['__PAD__'])
print("accuracy: ", accuracy)

outputs shape: (4,)
mask shape: (4,) mask: [ True  True False False]
accuracy:  1.0


In [268]:
def test_model(generator, model):
    '''
    Input: 
        generator: an iterator instance that provides batches of inputs and targets
        model: a model instance 
    Output: 
        accuracy: float corresponding to the accuracy
    '''
    
    accuracy = 0.
    total_num_correct = 0
    total_num_pred = 0
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    for batch in generator: 
        
        # Retrieve the inputs from the batch
        inputs = batch[0]
        
        # Retrieve the targets (actual labels) from the batch
        targets = batch[1]
        
        # Retrieve the example weight.
        example_weight = batch[2]

        # Make predictions using the inputs
        pred = model(inputs)
        
        # Calculate accuracy for the batch by comparing its predictions and targets
        batch_accuracy, batch_num_correct, batch_num_pred = compute_accuracy(pred, targets, example_weight)
        
        # Update the total number of correct predictions
        # by adding the number of correct predictions from this batch
        total_num_correct += batch_num_correct
        
        # Update the total number of predictions 
        # by adding the number of predictions made for the batch
        total_num_pred +=batch_num_pred

    # Calculate accuracy over all examples
    accuracy = total_num_correct / total_num_pred
    
    ### END CODE HERE ###
    return accuracy



In [272]:
model = training_loop.eval_model
accuracy = test_model(data_generator(val_pos, val_neg, 16, False, Vocab, shuffle), model)

print(f'The accuracy of your model on the validation set is {accuracy:.4f}', )

The accuracy of your model on the validation set is 0.5000


In [273]:
def predict(sentence):
    inputs = np.array(toTensor(sentence, vocabulary=Vocab))
    
    # Batch size 1, add dimension for batch, to work with the model
    inputs = inputs[None, :]  
    
    # predict with the model
    preds_probs = model(inputs)
    
    # Turn probabilities into categories
    preds = int(preds_probs[0, 1] > preds_probs[0, 0])
    
    sentiment = "negative"
    if preds == 1:
        sentiment = 'positive'

    return preds, sentiment

In [275]:
sentence = "i'm feeling good today"
tmp_pred, tmp_sentiment = predict(sentence)
print(f"The sentiment of the sentence \n***\n\"{sentence}\"\n***\nis {tmp_sentiment}.")

print()
# try a negative sentence
sentence = "I hated my day"
tmp_pred, tmp_sentiment = predict(sentence)
print(f"The sentiment of the sentence \n***\n\"{sentence}\"\n***\nis {tmp_sentiment}.")

The sentiment of the sentence 
***
"i'm feeling good today"
***
is positive.

The sentiment of the sentence 
***
"I hated my day"
***
is positive.
